# Tutorial 2: Algebraic Equations (Linear and Non-Linear)

In engineering problems (be it reactor design or process control), we are often tasked with finding a **model** of the system, which describes the dynamic physical/chemical/mechanical etc. relationships between the relevant variables.

The simplest models can be expressed using **algebraic equations**, which invoke simple mathematical operations on the variables, and no calculus such as differentiation or integration. These models can be further divided into **linear** and **non-linear** models as follows:

* **Linear** - involves only addition and subtraction of variables
* **Non-Linear** - involves polynomial, trigonometric, logarithmic, and-or inter-coupling between variables

The first type of system (**linear**) is relatively easy to solve using linear algebra and matrix manipulations. Let's discuss how to do this in Python.

## Problem 1: Solving linear systems of equations
Suppose we have the following three equations which describe a linear system between the variables $x_1$, $x_2$, and $x_3$:

$x_1+x_2+x_3 = 10$
$2x_1+x_2+x_3 = 5$
$-x_1+2x_2+4x_3 = 4$

In Math 152 (or any equivalent introductary linear algebra course) you may have learned to solve this using Gaussian elimination and back-substitution. However, all these steps can be easily bypassed by using Python's built-in functions:

$A = \begin{bmatrix} 1 & 1 & 1 \\
2 & 1 & 1 \\
-1 & 2 & 4 
\end{bmatrix}$,
$b = \begin{bmatrix} 10 \\
5 \\
4
\end{bmatrix}$

The solution is $A^{-1}b$ which we now compute.

In [56]:
# Import relevant packages
%matplotlib inline 
import matplotlib.pyplot as plt # for plots
import numpy as np # for most computations

In [57]:
# Define matrices
A = np.array([[1,1,1],[2,1,1],[-1,2,4]])
b = np.array([[10],[5],[4]])

# Find solution
x = np.linalg.solve(A,b)
print(x)

[[ -5. ]
 [ 30.5]
 [-15.5]]


That was pretty straightforward. As a sanity check, let's compute $Ax$ and see whether that indeed equals b:

In [58]:
print(np.dot(A,x))
print(b)

[[10.]
 [ 5.]
 [ 4.]]
[[10]
 [ 5]
 [ 4]]


Indeed the answer is correct as shown.

In some cases the $A$ matrix might be *non-invertible*. In other words, the *rank* of $A$ (i.e. number of independent rows or columns) is less than its maximum size. Consider the examples below:

$$A_1 = \begin{bmatrix} 1 & 1 & 1 \\
1 & 1 & 1 \\
-1 & 2 & 4 
\end{bmatrix}$$

$$A_2 = \begin{bmatrix} 1 & 1 & 1 \\
-1 & 2 & 4 
\end{bmatrix}$$

$$A_3 = \begin{bmatrix} 1 & 1 & 1 \\
2 & 1 & 1 \\
-1 & 2 & 4 \\
1 & 2 & 5
\end{bmatrix}$$

The matrix $A_1$ is non-invertible, as confirmed by evaluating its determinant and/or its eigenvalues:

In [59]:
A1 = np.array([[1,1,1],[1,1,1],[-1,2,4]])
print(np.linalg.det(A1))
print(np.linalg.eigvals(A1))

0.0
[-4.44089210e-16  1.58578644e+00  4.41421356e+00]


Recall that the determinant of a square matrix is the product of its eigenvalues. If any single eigenvalue is equal to zero (to the closest precision), then the matrix is non-invertible.

But what about matrices $A_2$ and $A_3$? The rank of matrix $A_2$ is 2, but it has 3 columns. Therefore it corresponds to an *underdefined* problem where there are more variables (3) than equations (2). The opposite is true for $A_3$ - this corresponds to an *overdefined* problem where there are more equations than variables, unless one of the equations is a linear combination of the remainder. 

Although the inverse of a non-square matrix is undefined, we can still compute solutions to these problems using a neat trick, something similar to an inverse. This known as the **Moore-Penrose Pseudoinverse** (or *general pseudoinverse*), represented by the compact *dagger* notation:

$$A^{\dagger}\triangleq (A^{\top}A)^{-1}A^{\top}$$

For example, if we take:
$$A_2 = \begin{bmatrix} 1 & 1 & 1 \\
-1 & 2 & 4 
\end{bmatrix}$$ and

$$b_2 = \begin{bmatrix} 10 \\
4
\end{bmatrix}$$

we can still compute *a* solution (out of the infinitely-many solutions) to this problem, known as the *least-squares* solution, using the **psueodinverse**:

$$x_{2,LS}=A_2^{\dagger}b$$

This is known as the *least-squares* solution, because it is equivalent to finding the *weights* of the linear regression model between the two datapoints $X = \begin{bmatrix} 1 & 1 & 1 \\
-1 & 2 & 4 
\end{bmatrix}$ and two corresponding outputs $y=\begin{bmatrix} 10 \\
4
\end{bmatrix}$.

Now let's perform this psuedoinverse operation in python:

In [60]:
A2 = np.array([[1,1,1],[-1,2,4]])
b = np.array([[10],[4]])
A2_dagger = np.linalg.pinv(A2)
x2_LS = np.dot(A2_dagger,b)
print(x2_LS)

[[6.]
 [3.]
 [1.]]


Let's again perform our usual sanity check to confirm this is indeed a solution:

In [61]:
print(np.dot(A2,x2_LS))
print(b)

[[10.]
 [ 4.]]
[[10]
 [ 4]]


As the last example, let's perform the psueodinverse operation on matrix $A_3$ and see whether an *exact* solution exists. If no *exact* solution exists, then the problem is *overdefined*, i.e. there are four equations explaining three variables, with one equation contradicting the rest.

In [62]:
A3 = np.array([[1,1,1],[2,1,1],[-1,2,4],[1,2,5]])
b = b = np.array([[10],[5],[4],[10]])
A3_dagger = np.linalg.pinv(A3)
x3_LS = np.dot(A3_dagger,b)

Now let's see if $A_3 x_{3,LS}$ gives the exact values for $b$:

In [63]:
print(np.dot(A3,x3_LS))
print(b)

[[6.03496503]
 [8.08391608]
 [5.32167832]
 [9.11888112]]
[[10]
 [ 5]
 [ 4]
 [10]]


Clearly, it doesn't, therefore $A_3$ represents an *overdefined* problem.

# Problem 2: Solving systems of non-linear equations

Imagine now that you have an engineering problem which boils down to the following equations:

$$x_1 x_2-log_{10}(x_1)=1$$
$$x_1^2+x_2^2=100.04$$

This is a **non-linear** system, because it involves the variable-coupling term $x_1 x_2$ as well as non-linear functions of both $x_1$ and $x_2$ individually - specifically, $log_{10}(x_1)$, $x_1^2$, and $x_2^2$.

Matrix inverses and psuedo-inverses are useless here, since those only work for linear systems. So how do we solve this? 

Fortunately, in your introductory numerical operations course (CHBE230 or equivalent), you learned about the technique of **root-finding**. So let's transform the previous problem into a suitable form for root-finding.

First, we re-write the problem as:
$$x_1 x_2-log_{10}(x_1)-1=0$$
$$x_1^2+x_2^2-100.04=0$$

Then we define a function $F=\begin{bmatrix}f_1 \\ f_2 \end{bmatrix}$ acting on both variables $x=\begin{bmatrix}x_1 \\ x_2 \end{bmatrix}$ such that:
$$F(x)=\begin{bmatrix}f_1 \bigg(\begin{bmatrix}x_1 \\ x_2 \end{bmatrix} \bigg) \\ f_2 \bigg(\begin{bmatrix}x_1 \\ x_2 \end{bmatrix} \bigg) \end{bmatrix}=0$$

Now we can see that $f_1$ and $f_2$ must equal to the re-written expressions above. We can now root-find this 2-dimensional function $F$ using many methods, including Newton's method, Broyden, Krylov, etc. Fortunately, *scipy* has a built-in function to do this, so let's use it:

In [83]:
# First we define big F, and recall that Python starts indices at 0 and not 1!
def bigF(x):
    return np.array([x[0]*x[1] - np.log10(x[0]) - 1.0,np.power(x[0],2) + np.power(x[1],2) - 100.04])

In [87]:
# Now let's find the solution:
x_init = np.array([[1],[1]]) # Initialization

from scipy import optimize
sol = optimize.root(bigF, x_init, method='hybr')
x_sol=sol.x

As usual, let's perform a sanity check on these solutions, to ensure they evaluate to zero (within precision):

In [89]:
bigF(x_sol)

array([5.77924686e-09, 1.20848910e-07])

That's all there is to non-linear systems! Note that the cases of *underdefined* and *overdefinde* still exist, but neither possess well-defined/recognizable forms as in the case of linear systems.